## Market1501 / cuhk03 

In [1]:
%matplotlib inline
import json
from pprint import pprint
Settings = json.load(open('settings.txt'))
pprint(Settings)
print('')
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../')
from reid.stacknet import get_model
from reid.data import DataSampler

from keras.callbacks import ModelCheckpoint, TerminateOnNaN
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
from os.path import join, isfile, isdir, exists, splitext

root = Settings['data_root']

sampler = DataSampler(root, 221, 221)

x, y = sampler.get_test_batch(5, 5)

pid = 3
print('is same:', y[pid])
im = x[pid]
fig = plt.figure(figsize=(10, 6))
fig.add_subplot(121).imshow(im[:,:,0:3])
fig.add_subplot(122).imshow(im[:,:,3:6])
plt.show()

{'UMPM': {'password': 'ahJaka4o', 'username': 'umpmuser'},
 'data_root': '/home/DATA'}



Using TensorFlow backend.


/home/DATA/cuhk03_release found :)
could not find folder /home/DATA/Market-1501-v15.09.15...
found /home/DATA/Market-1501-v15.09.15.zip
unzip /home/DATA/Market-1501-v15.09.15.zip -> /home/DATA/Market-1501-v15.09.15
unzip exception:Bad magic number for file header
	try unzip using OS tools
	unzip using OS tools
	err:	None
/home/DATA/DukeMTMC-reID found :)
found memmaped /home/DATA/cuhk03_release/labeled221x221.npy
(cuhk) positive test pairs: 9290
(cuhk) positive train pairs: 127242


OSError: cannot identify image file '/home/DATA/Market-1501-v15.09.15/Market-1501-v15.09.15/bounding_box_test/-1_c2s1_030076_06.jpg'

## UMPM dataset

In [ ]:
from time import time
from reid.data import UMPMSampler

root = Settings['data_root']
print(root)

user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']
umpm_sampler = UMPMSampler(root, ['p2_chair_2'], user, pwd, 221, 221)


_start = time()
x, y = umpm_sampler.get_train()
_end = time()
print('elapsed', _end - _start)

print('same', y[0])

print(x.shape)

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(121)
ax.imshow(x[0, :, :, 0:3].astype('uint8'))
ax = fig.add_subplot(122)
ax.imshow(x[0, :, :, 3:6].astype('uint8'))
plt.show()

## MOT16 dataset

In [ ]:
from reid.data import MOT16Sampler

mot_sampler = MOT16Sampler(root, 221, 221)


_start = time()
x, y = mot_sampler.sample()
_end = time()
print('elapsed', _end - _start)

print('same', y[0])

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(121)
ax.imshow(x[0,:,:,0:3].astype('uint8'))
ax = fig.add_subplot(122)
ax.imshow(x[0,:,:,3:6].astype('uint8'))
plt.show()

## Training

In [ ]:
from keras.callbacks import ModelCheckpoint, TerminateOnNaN
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input

filepath = join('data_storage', 'model.h5')

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, TerminateOnNaN()]

if isfile(filepath):
    model = load_model(filepath)
else:
    model = get_model()

model.summary()

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import RMSprop
from keras import backend as K


dn = DenseNet121(weights='imagenet', input_shape=(221, 221, 3),
    include_top=False, pooling='max')


# input_a = Input(shape=(64, 64, 3))
# input_b = Input(shape=(64, 64, 3))

# processed_a = dn(input_a)
# processed_b = dn(input_b)

In [ ]:
# def generate_training_data():
#     global sampler
#     while True:
#         X, Y = sampler.get_train_batch(16, 16)
#         X = preprocess_input(X.astype('float64'))
#         yield X, Y

# def generate_validation_data():
#     global sampler
#     while True:
#         X, Y = sampler.get_test_batch(16, 16)
#         X = preprocess_input(X.astype('float64'))
#         yield X, Y

# history = model.fit_generator(generate_training_data(),
#                     validation_data=generate_validation_data(),
#                     validation_steps=5,
#                     steps_per_epoch=100,
#                     epochs=1000,
#                     callbacks=callbacks_list)